<a href="https://colab.research.google.com/github/MedwillTech/MedwillTech/blob/main/NotionEngeneering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Recap sur Ratissage du Web

Analyse syntaxique duHTM

In [ ]:
# !python -m pip install beautifulsoup4 requests html5lib
"""
 <html>
  <head>
    <title>A web page</title>
  </head>
  <body>
    <p id="author">Joel Grus</p>
    <p id="subject">Data Science</p>
  </body>
 </html>

"""
from bs4 import BeautifulSoup
import requests
 # Je place le fichier HTML qui nous occupe sur GitHub. Pour adapter
 # l’URL au format de l’ouvrage, je dois la répartir sur deux lignes.
 # Rappelez-vous que les chaînes séparées par des caractères d’espacement
 # sont concaténées.
url = ("https://raw.githubusercontent.com/"
       "joelgrus/data/master/getting-data.html")
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

print(soup.contents)

['html', '\n', <html lang="en-US">
<head>
<title>Getting Data</title>
<meta charset="utf-8"/>
</head>
<body>
<h1>Getting Data</h1>
<div class="explanation">
        This is an explanation.
    </div>
<div class="comment">
        This is a comment.
    </div>
<div class="content">
<p id="p1">This is the first paragraph.</p>
<p class="important">This is the second paragraph.</p>
</div>
<div class="signature">
<span id="name">Joel</span>
<span id="twitter">@joelgrus</span>
<span id="email">joelgrus-at-gmail</span>
</div>
</body>
</html>, '\n']


In [ ]:
print(soup.p.text.split())
print(soup.find_all("p"))

important_paragraphs = soup('p', {'class' : 'important'})
important_paragraphs2 = soup('p', 'important')
important_paragraphs3 = [p for p in soup('p')
                         if 'important' in p.get('class', [])]

print(important_paragraphs)
print(important_paragraphs2)
print(important_paragraphs3)

# Par exemple, si vous cherchez tous les éléments <span> contenus dans un élément
# <div>, procédez commesuit:

 # Attention, retourne le même <span> plusieurs fois de suite
 # s'il est présent dans plusieurs <div>.
 # soyez plus malin dans ce cas
spans_inside_divs = [span
                 # pour chaque <div> sur la page
                 for div in soup('div')
                 # trouver chaque <span> à l’intérieur
                 for span in div('span')]


print(spans_inside_divs)

['This', 'is', 'the', 'first', 'paragraph.']
[<p id="p1">This is the first paragraph.</p>, <p class="important">This is the second paragraph.</p>]
[<p class="important">This is the second paragraph.</p>]
[<p class="important">This is the second paragraph.</p>]
[<p class="important">This is the second paragraph.</p>]
[<span id="name">Joel</span>, <span id="twitter">@joelgrus</span>, <span id="email">joelgrus-at-gmail</span>]


**Exemple: Congrès sous surveillance**

Le vice-président chargé des politiques chez DataSciencester s’inquiète d’une potentielle réglementation du secteur de la science des données. Il vous demande de quantifier ce que le Congrès a à dire sur le sujet. Il veut plus particulièrement que vous identifiiez tous les repré sentants de cette institution des États-Unis qui sont associés à des communiqués de presse
relatifs aux données.
Au moment où nous publions ces lignes, une page contient des liens vers tous les sites web des représentants: https://www.house.gov/representatives.
Et si vous consultez la source, vous constatez que tous les liens vers ces sites web se présentent selon l’exemple suivant:

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# URL de la page des représentants
url = "https://www.house.gov/representatives"
text = requests.get(url).text

# Utilisation du parseur HTML standard de Python
soup = BeautifulSoup(text, "html.parser")

# Trouver tous les liens <a> qui contiennent les informations sur les représentants
# Nous allons chercher des liens qui sont typiquement dans des balises <a> de la liste des représentants
all_urls = [a['href'] for a in soup.find_all('a', href=True)]

# Affichage du nombre total de liens
print(f"Total des URLs trouvées : {len(all_urls)}")

# Expression régulière pour filtrer les liens .house.gov
regex = r"^https?://.*\.house\.gov/?$"
good_urls = [url for url in all_urls if re.match(regex, url)]

# Suppression des doublons en utilisant un set
good_urls = list(set(good_urls))

# Affichage du nombre de bonnes URLs
print(f"Nombre d'URLs filtrées (représentants) : {len(good_urls)}")

# Extraction des noms des représentants à partir des bonnes URLs
representatives = []
for house_url in good_urls:
    html = requests.get(house_url).text
    soup = BeautifulSoup(html, 'html.parser')

    # Recherche des noms des représentants dans des éléments <h1> ou autres balises pertinentes
    name = soup.find('h1')  # Habituellement, les noms des représentants sont dans <h1> ou <title>
    if name:
        representatives.append(name.get_text(strip=True))

# Affichage des 20 premiers noms de représentants
print("Les 20 premiers représentants trouvés :")
for i, rep in enumerate(representatives[:20]):
    print(f"{i + 1}. {rep}")

# Si vous souhaitez plus de détails sur chaque représentant (exemple : leurs liens de presse)
press_releases = {}
for house_url in good_urls[:20]:  # Limiter aux 20 premiers pour éviter trop de requêtes
    html = requests.get(house_url).text
    soup = BeautifulSoup(html, 'html.parser')

    # Recherche des liens de presse sur la page du représentant
    pr_links = {a['href'] for a in soup.find_all('a', href=True) if 'press releases' in a.text.lower()}
    press_releases[house_url] = pr_links

# Affichage des liens de presse pour les 20 premiers
for house_url, pr_links in press_releases.items():
    print(f"\nLiens de presse pour {house_url}:")
    for pr_link in pr_links:
        print(f" - {pr_link}")




Total des URLs trouvées : 967
Nombre d'URLs filtrées (représentants) : 439
Les 20 premiers représentants trouvés :
1. Home
2. ICYMI: Stefanik Highlights House Republicans' Oversight Uprooting Corruption: "It's Our Job to Get Transparency and Accountability"
3. Home
4. Home
5. U.S. Congressman French Hill. Serving the 2nd District of Arkansas.
6. Home
7. Proudly Serving theVirgin Islands of the United States
8. Home
9. Home
10. Home
11. Home
12. How Can I Help?
13. PROUDLYSERVING FLORIDA
14. REPORT & VIDEO: Congresswoman Ross Releases 2024 Year-End Report
15. Home
16. Let's Stay Connected
17. David Scott. Representing Georgia's 13th District
18. Home
19. Representing New York's 26th District
20. Hi, I'mRep. Gluesenkamp Perez

Liens de presse pour https://lofgren.house.gov/:
 - /media/press-releases

Liens de presse pour https://ansari.house.gov/:
 - /media/press-releases

Liens de presse pour https://mrvan.house.gov:
 - /media/press-releases

Liens de presse pour https://stefanik.house.

**Recap sur les Fichiers à délimiteur**

In [ ]:
# exemple de datei csv ou txt
# Nom du fichier est: tab_delimited_stock_prices.txt

6/20/2014 AAPL 90.91
6/20/2014 MSFT 41.68
6/20/2014 FB 64.5
6/19/2014 AAPL 91.86
6/19/2014 MSFT 41.51
6/19/2014 FB 64.34

import csv
with open('tab_delimited_stock_prices.txt') as f:
  tab_reader = csv.reader(f, delimiter='\t') # on peut avoir aussi  delimiter=':' ou '-' ou ',' etc...
  for row in tab_reader:
    date = row[0]
    symbol = row[1]
    closing_price = float(row[2])
    # traitement des données...


**Recap sur la Lecture de fichiers**

In [ ]:
def get_domain(email_address: str) -> str:
  """couper au niveau de '@' et retourner le dernier morceau"""
  return email_address.lower().split("@")[-1]

 # deux tests
assert get_domain('joelgrus@gmail.com') == 'gmail.com'
assert get_domain('joel@m.datasciencester.com') == 'm.datasciencester.com'

from collections import Counter
with open('email_addresses.txt', 'r') as f:
  domain_co unts = Counter(get_domain(line.strip())
                           for line in f
                           if "@" in line)

**Recap sur les: "stdin" et "stdout"**

In [ ]:
# en Bash: cat output.txt | python test.py
# en Windows: type output.txt | python test.py
# otput.txt--> contient le text
# test.üy---> contient le bout de code python
import sys
from collections import Counter

 # passer le nombre de mots en premier argument
try:
    num_words = int(sys.argv[1])
except:
    print("usage: test.py num_words")
    sys.exit(1)  # un code non zéro en sortie indique une erreur
counter = Counter(word.lower()  # mots en minuscules
                  for line in sys.stdin
                  # séparer à partir des espaces
                  for word in line.strip().split()
                  if word)  # sauter les mots (words) vides
for word, count in counter.most_common(num_words):
  sys.stdout.write(str(count))
  sys.stdout.write("\t")
  sys.stdout.write(word)
  sys.stdout.write("\n")